In [11]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import os
import numpy as np
import cv2
import re


In [9]:
## checking crs

with rasterio.open("5b1b6fb2-5024-4681-a175-9b667174f48c.tif") as src:
    print("Raster CRS:", src.crs)

tiles = gpd.read_file("tiles.geojson")
print("Tiles CRS:", tiles.crs)


Raster CRS: PROJCS["unnamed",GEOGCS["unnamed ellipse",DATUM["unknown",SPHEROID["unnamed",6378137,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Mercator_2SP"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Tiles CRS: EPSG:4326


In [10]:
## tile the tif image into smaller patches

import rasterio
import geopandas as gpd
from rasterio.mask import mask
import os

raster_path = "/Users/saharmohamed/Documents/UBS_Material/DL4CV/Project_Yolo/image_raster.tif"
tiles_path = "tiles.geojson"
tif_out_dir = "images_tif"

os.makedirs(tif_out_dir, exist_ok=True)

tiles = gpd.read_file(tiles_path)


with rasterio.open(raster_path) as src:
    tiles = tiles.to_crs(src.crs)
    for _, tile in tiles.iterrows():
        geom = [tile.geometry]
        tile_id = tile["id"]  

        out_image, out_transform = mask(
            src,
            geom,
            crop=True
        )

        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        out_path = os.path.join(
            tif_out_dir,
            f"{tile_id}.tif"
        )

        with rasterio.open(out_path, "w", **out_meta) as dst:
            dst.write(out_image)


In [13]:
## convert tif to png

def tif_to_png(tif_path, png_path):
    with rasterio.open(tif_path) as src:
        img = src.read([1, 2, 3])  # RGB
        img = np.transpose(img, (1, 2, 0))
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        img = img.astype("uint8")

    cv2.imwrite(png_path, img)

In [14]:
png_out_dir = "images_png"
os.makedirs(png_out_dir, exist_ok=True)

for fname in os.listdir("images_tif"):
    if fname.endswith(".tif"):
        tif_path = os.path.join("images_tif", fname)
        png_path = os.path.join(
            png_out_dir,
            fname.replace(".tif", ".png")
        )
        tif_to_png(tif_path, png_path)

In [15]:
for fname in os.listdir("images_png"):
    if fname.endswith(".png"):
        # extract x,y,z
        xyz = re.search(r"x=(\d+), y=(\d+), z=(\d+)", fname)
        if xyz:
            x, y, z = xyz.groups()
            new_name = f"OAM-{x}-{y}-{z}.png"
            os.rename(
                os.path.join("images_png", fname),
                os.path.join("images_png", new_name)
            )

In [20]:
def draw_yolo_boxes(image_path, label_path, out_path):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    img = cv2.imread(image_path)
    h, w, _ = img.shape

    if not os.path.exists(label_path):
        cv2.imwrite(out_path, img)
        return

    with open(label_path) as f:
        for line in f:
            cls, xc, yc, bw, bh = map(float, line.split())

            x1 = int((xc - bw / 2) * w)
            y1 = int((yc - bh / 2) * h)
            x2 = int((xc + bw / 2) * w)
            y2 = int((yc + bh / 2) * h)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.imwrite(out_path, img)





In [22]:
draw_yolo_boxes(
    image_path="images_png/OAM-6773-293582-19.png",
    label_path="labels/yolo/OAM-6773-293582-19.txt",
    out_path="debug/check.png"
)